In [ ]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import hashlib
import math
import os
import random
import sys
from tempfile import gettempdir
import zipfile
import pickle
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector

data_index = 0


import pandas as pd

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02" -O imdb.csv


--2020-09-19 01:35:31--  https://docs.google.com/uc?export=download&confirm=&id=1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02
Resolving docs.google.com (docs.google.com)... 74.125.203.139, 74.125.203.102, 74.125.203.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.203.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-1k-docs.googleusercontent.com/docs/securesc/jmhctunv9qd75m91b3g2ql5qjatrq53u/jif28qqeubdp4a1q4du8467j7bi0cbqh/1600479300000/00519734162681609028/09820975160179898913Z/1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02?e=download [following]
--2020-09-19 01:35:36--  https://doc-00-1k-docs.googleusercontent.com/docs/securesc/jmhctunv9qd75m91b3g2ql5qjatrq53u/jif28qqeubdp4a1q4du8467j7bi0cbqh/1600479300000/00519734162681609028/09820975160179898913Z/1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02?e=download
Resolving doc-00-1k-docs.googleusercontent.com (doc-00-1k-docs.googleusercontent.com)... 74.125.204.132, 2404:6800:4008:c04::84
Connecting to 

In [ ]:
import io
dados = pd.read_csv('imdb.csv')

In [ ]:
dados.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
def tokens(data): #utilizado para separar as frases em palavras separadas
    ret = []
    for s in data.iterrows():
        ret +=s[1][0].split()
    return ret

def build_dataset(words, n_words): #criar o dataset em tensorflow
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1)) #pega as n_words mais comuns, evita entao ter palavras bizarras
    dictionary = {word: index for index, (word, _) in enumerate(count)} #dicionario para fazer a conversao de int - palavra
    data = []
    unk_count = 0
    for word in words:
      index = dictionary.get(word, 0)
      if index == 0:  # dictionary['UNK']
        unk_count += 1
      data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [ ]:
vocabulary_size = 40000
fix_break = lambda x: x.replace("<br />"," <br/> ")
fix_dots = lambda x: x.replace("."," . ")
fix_comma = lambda x: x.replace(","," , ")
fix_brackets1 = lambda x: x.replace("("," ( ")
fix_brackets2 = lambda x: x.replace(")"," ) ")
fix_dash = lambda x: x.replace("-"," - ")     
fix_excl = lambda x: x.replace("!"," ! ")     
dados.review = dados.review.apply(fix_break)
dados.review = dados.review.apply(fix_dots)
dados.review = dados.review.apply(fix_comma)
dados.review = dados.review.apply(fix_brackets1)
dados.review = dados.review.apply(fix_brackets2)
dados.review = dados.review.apply(fix_excl)
dados.review = dados.review.apply(fix_dash)

In [ ]:
tok = tokens(dados)


In [ ]:
data, count, unused_dictionary, reverse_dictionary = build_dataset(tok, vocabulary_size)

In [ ]:
def generate_batch(batch_size, num_skips, skip_window): #gera a batch para um formato que o tensorflow entende
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # o tamanho da janela de previsao do skio
    buffer = collections.deque(maxlen=span)  # magia negra que precisei colocar, sei la pra que serve saporra mas soh funciona com isso
    if data_index + span > len(data):
      data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
      context_words = [w for w in range(span) if w != skip_window]
      words_to_use = random.sample(context_words, num_skips)
      for j, context_word in enumerate(words_to_use):
        batch[i * num_skips + j] = buffer[skip_window]
        labels[i * num_skips + j, 0] = buffer[context_word]
      if data_index == len(data):
        buffer.extend(data[0:span])
        data_index = span
      else:
        buffer.append(data[data_index])
        data_index += 1
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [ ]:
batch_size = 128 
embedding_size = 256  # Dimensao do embedding vector. Hyperparametro que pode ser mudado e ajustado para cada modelo
skip_window = 1  # A janela ja referida anteriormente
num_skips = 2  # quantas vezes o label vai ser reusado
num_sampled = 64  # Para o negative sampling, usando 64 pq eh o recomendado.

valid_size = 16  # As palavras quse seram utilziadas para avaliar a similaridade.
valid_window = 100  #janeal para a sampling das palavras de similaridade.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

graph = tf.Graph()

In [ ]:
current_path = os.path.dirname(os.path.realpath(sys.argv[0])) #formalidadaes para rodar o codigo, meio inutil
parser = argparse.ArgumentParser()
parser.add_argument(
      '--log_dir',
      type=str,
      default=os.path.join(current_path, 'log'),
      help='The log directory for TensorBoard summaries.')
flags, unused_flags = parser.parse_known_args()

log_dir = flags.log_dir

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [ ]:
with graph.as_default():
    # Input data
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    # Utilizando CPU pq tava fazendo no meu mac, nao tem tanto ganho assim com gpu ai deixei assim mesmo
    with tf.device('/cpu:0'):
        # Ver os embeddings na matriz
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
      # Constroi as variaveis para a NCE loss
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                tf.truncated_normal([vocabulary_size, embedding_size],stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocabulary_size))
    # Adiciona o valor da los
    tf.summary.scalar('loss', loss)
    # COnstroi o optimizer usando gradient descnete e uma learning rate de 1.0.
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    # Calcula a similaridade de cada embedding
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)
    # Junta td
    merged = tf.summary.merge_all()
    # INicializador
    init = tf.global_variables_initializer()
    # Meio inutil, codigo vestigial
    saver = tf.train.Saver()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
num_steps = 500001


In [ ]:
with tf.compat.v1.Session(graph=graph) as session:
    # Gambiarra do tensorflow, abre esse writer para escrever em um arquivo para o futuro
    writer = tf.summary.FileWriter(log_dir, session.graph)
    # inicializa as variaveis
    init.run()
    print('Initialized')
    average_loss = 0
    for step in xrange(num_steps):
      batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                  skip_window)
      feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
      # Mais uma magia negra do tensorflow, define a metadata
      run_metadata = tf.RunMetadata()

      _, summary, loss_val = session.run([optimizer, merged, loss],
                                         feed_dict=feed_dict,
                                         run_metadata=run_metadata)
      average_loss += loss_val
      writer.add_summary(summary, step)

      if step == (num_steps - 1):
        writer.add_run_metadata(run_metadata, 'step%d' % step)
      if step % 2000 == 0:
        if step > 0:
          average_loss /= 2000
 
        print('Average loss at step ', step, ': ', average_loss)
        average_loss = 0

      if step % 10000 == 0:
        sim = similarity.eval()
        for i in xrange(valid_size):
          valid_word = reverse_dictionary[valid_examples[i]]
          top_k = 8  # number of nearest neighbors
          nearest = (-sim[i, :]).argsort()[1:top_k + 1]
          log_str = 'Nearest to %s:' % valid_word
          print(
              log_str,
              ', '.join([reverse_dictionary[nearest[k]] for k in range(top_k)]))
    final_embeddings = normalized_embeddings.eval()

    with open(log_dir + '/metadata.tsv', 'w') as f:
      for i in xrange(vocabulary_size):
        f.write(reverse_dictionary[i] + '\n')

    saver.save(session, os.path.join(log_dir, 'model.ckpt'))
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join(log_dir, 'metadata.tsv')
    projector.visualize_embeddings(writer, config)


In [ ]:
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pickle
pickle.dump( final_embeddings, open( "embeds_imdb.p", "wb" ) )
pickle.dump( unused_dictionary, open( "dictionary_imdb.p", "wb" ) )


In [ ]:
model_file = drive.CreateFile({'title' : 'embeds_imdb.p'})
model_file.SetContentFile('./embeds_imdb.p')
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '17CszXuAvtdvUfxWtvlrdAUGhkW1LPPbe'})

In [ ]:
model_file2 = drive.CreateFile({'title' : 'dictionary_imdb.p'})
model_file2.SetContentFile('./dictionary_imdb.p')
model_file2.Upload()
drive.CreateFile({'id': model_file2.get('id')})

GoogleDriveFile({'id': '1_uny6na4DiXoUPB7eIZBDkFNNJIjFV5y'})

# Model stuff

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02" -O imdb.csv
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=17CszXuAvtdvUfxWtvlrdAUGhkW1LPPbe' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=17CszXuAvtdvUfxWtvlrdAUGhkW1LPPbe" -O embeds_imdb.p
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_uny6na4DiXoUPB7eIZBDkFNNJIjFV5y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1_uny6na4DiXoUPB7eIZBDkFNNJIjFV5y" -O dictionary_imdb.p


--2020-09-21 00:14:09--  https://docs.google.com/uc?export=download&confirm=&id=1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02
Resolving docs.google.com (docs.google.com)... 108.177.119.102, 108.177.119.139, 108.177.119.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.119.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-40-docs.googleusercontent.com/docs/securesc/819i5sl1ttha60t5qkt7442uoue3528p/fmb33lijsav734dl3qcss2f9sv33lgg0/1600647225000/00519734162681609028/05482464812705070839Z/1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02?e=download [following]
--2020-09-21 00:14:10--  https://doc-0c-40-docs.googleusercontent.com/docs/securesc/819i5sl1ttha60t5qkt7442uoue3528p/fmb33lijsav734dl3qcss2f9sv33lgg0/1600647225000/00519734162681609028/05482464812705070839Z/1H-xa2AqlvDfBdKN6QJVKbnP-Aw7wzZ02?e=download
Resolving doc-0c-40-docs.googleusercontent.com (doc-0c-40-docs.googleusercontent.com)... 172.217.218.132, 2a00:1450:4013:c08::84
Connectin

In [3]:
import pickle
import pandas as pd
embeds = pickle.load(open("embeds_imdb.p","rb"))
imdb_dict = pickle.load(open("dictionary_imdb.p","rb"))

In [4]:
dados = pd.read_csv('imdb.csv')
fix_break = lambda x: x.replace("<br />"," <br/> ")
fix_dots = lambda x: x.replace("."," . ")
fix_comma = lambda x: x.replace(","," , ")
fix_brackets1 = lambda x: x.replace("("," ( ")
fix_brackets2 = lambda x: x.replace(")"," ) ")
fix_dash = lambda x: x.replace("-"," - ")     
fix_excl = lambda x: x.replace("!"," ! ")     
dados.review = dados.review.apply(fix_break)
dados.review = dados.review.apply(fix_dots)
dados.review = dados.review.apply(fix_comma)
dados.review = dados.review.apply(fix_brackets1)
dados.review = dados.review.apply(fix_brackets2)
dados.review = dados.review.apply(fix_excl)
dados.review = dados.review.apply(fix_dash)

In [5]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
class SimpleLSTM(nn.Module):
    def __init__(self,size,hidden_size,num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size = size,hidden_size = hidden_size, num_layers = 1, bidirectional = True)
        self.output_layer = nn.Linear(2*hidden_size,2)
        self.activation = F.relu
    def forward(self,inputs,seq_len):
        import pdb;pdb.set_trace()
        packed_input = pack_padded_sequence(inputs.float(),seq_len,enforce_sorted = False)
        _, (h_n, c_n) = self.lstm(packed_input)
        lstm_output = torch.cat([h_n[-1], h_n[-2]], -1)
        output = self.output_layer(self.activation(lstm_output))
        return output


In [6]:
!pip install ujson

In [7]:
import dill as dill
import os
import ujson
from torch.utils.data import Dataset, DataLoader
import json
from smart_open import open
from sklearn.utils import shuffle
import msgpack
import io
from collections import Counter
import numpy as np
import torch
import pickle
from torch.nn.utils.rnn import pad_sequence
import random

from functools import lru_cache


In [8]:
def convert_embeddings(lookup,word_dict,words):
    #import pdb;pdb.set_trace()
    string_list = words.split(" ")
    vectors = []
    for x in string_list:
        if x in word_dict.keys():
            vectors.append(lookup[word_dict[x]])
        else:
            vectors.append(lookup[word_dict["UNK"]])
    return np.array(vectors)


In [9]:
from tqdm.notebook import tqdm
import io
from collections import Counter
import pickle
from smart_open import open
import msgpack
#import msgpack_numpy as m
#m.patch()
def pandas_to_bytes(df):
    buf = io.BytesIO()
    df.to_msgpack(path_or_buf = buf)
    return buf.getvalue()
def bytes_to_pandas(content):
    buf = io.BytesIO(content)
    return pd.read_msgpack(buf)
def tensor_to_bytes(tensor):
    buf = io.BytesIO()
    torch.save(tensor, buf)
    return buf.getvalue()
def bytes_to_tensor(content):
    buf = io.BytesIO(content)
    return torch.load(buf)

In [10]:
"""
data = dados
train = []
test = []
fname2 = f'imdb_tensors_train.msgpack.gz'
fname3 = f'imdb_tensors_test.msgpack.gz'
g = open(fname2, 'wb')
h = open(fname3, 'wb')
for ind,row in tqdm(dados.iterrows()):
    txt = convert_embeddings(embeds,imdb_dict,row.review)
    if row.sentiment == "positive":
        label = 1

    else:
        label = 0
    if ind%5==0:
        msgpack.dump([tensor_to_bytes(txt),label],h)
    else:
        msgpack.dump([tensor_to_bytes(txt),label],g)

g.close()
h.close()
"""

'\ndata = dados\ntrain = []\ntest = []\nfname2 = f\'imdb_tensors_train.msgpack.gz\'\nfname3 = f\'imdb_tensors_test.msgpack.gz\'\ng = open(fname2, \'wb\')\nh = open(fname3, \'wb\')\nfor ind,row in tqdm(dados.iterrows()):\n    txt = convert_embeddings(embeds,imdb_dict,row.review)\n    if row.sentiment == "positive":\n        label = 1\n\n    else:\n        label = 0\n    if ind%5==0:\n        msgpack.dump([tensor_to_bytes(txt),label],h)\n    else:\n        msgpack.dump([tensor_to_bytes(txt),label],g)\n\ng.close()\nh.close()\n'

In [11]:
"""from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""

'from pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n'

In [12]:
"""
model_file = drive.CreateFile({'title' : 'imdb_tensors.msgpack.gz'})
model_file.SetContentFile('./imdb_tensors.msgpack.gz')
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})"""

"\nmodel_file = drive.CreateFile({'title' : 'imdb_tensors.msgpack.gz'})\nmodel_file.SetContentFile('./imdb_tensors.msgpack.gz')\nmodel_file.Upload()\ndrive.CreateFile({'id': model_file.get('id')})"

In [13]:
!ls -l --block-size=MB

total 130MB
-rw-r--r-- 1 root root  1MB Sep 21 00:14 dictionary_imdb.p
-rw-r--r-- 1 root root 41MB Sep 21 00:14 embeds_imdb.p
-rw-r--r-- 1 root root 67MB Sep 21 00:14 imdb.csv
drwxr-xr-x 1 root root  1MB Sep 16 16:29 sample_data
-rw-r--r-- 1 root root 11MB Sep 21 00:05 test_best_model.pth
-rw-r--r-- 1 root root 11MB Sep 21 00:05 test_last_model.pth


In [14]:
!ls

dictionary_imdb.p  imdb.csv	test_best_model.pth
embeds_imdb.p	   sample_data	test_last_model.pth


In [15]:
class ImdbDataset(Dataset):
    def __init__(self, num_lines, train=False):
        self.fname = f'imdb_tensors_train.msgpack.gz' if train else f'imdb_tensors_test.msgpack.gz'
        self.len = 0
        #import pdb;pdb.set_trace()
        with open(self.fname, 'rb') as g:
            tmp = msgpack.Unpacker(g)
            for n,i in tqdm(enumerate(tmp)):
                self.len+=1
        self.f= open(self.fname, 'rb')
        self.sections = msgpack.Unpacker(self.f)
    def __len__(self):
        return self.len
    def __getitem__(self, item):
        for n, section in enumerate(self.sections):
            txt = bytes_to_tensor(section[0])
            label = section[1]
            break     
        return (txt,label)

In [16]:
class ImdbDataset_v2(Dataset):
    def __init__(self,dados, num_lines, train=False):
        self.len = 0
        if train:
            self.data = dados.sample(frac=1).reset_index(drop=True)
        else:
            self.data = dados.iloc[:10000].sample(frac=1).reset_index(drop=True)
        #import pdb;pdb.set_trace()
    def __len__(self):
        return len(self.data)
    def __getitem__(self, item):
        tmp = self.data.iloc[item]
        txt = convert_embeddings(embeds,imdb_dict,tmp.review)
        if tmp.sentiment == "positive":
            label = 1

        else:
            label = 0
        return (txt,label)

In [17]:
def pad_batch(batch):
    #import pdb;pdb.set_trace()
    element_vectors, targets = zip(*batch)
    num_elements = torch.tensor([len(c) for c in element_vectors], dtype=torch.long)#long
    element_tensors =[torch.tensor(x) for x in element_vectors] 
    tensor_out = pad_sequence(element_tensors)
    targets = torch.tensor(targets)
    return (
        (tensor_out, num_elements),
        targets
    )

In [18]:
def set_seeds(worker_id):
    seed = torch.initial_seed() % 2 ** 31
    np.random.seed(seed + 1)
    random.seed(seed + 2)

In [19]:
import dill as dill
import os
import ujson
BASE_PATH = "/home/ec2-user/SageMaker/DataTeamExperiments/Dasa-Impar-Embedding"
OBSERVER_PATH = os.path.join(BASE_PATH, "Model_Results")
TRAIN = "train"
VALIDATION = "validation"
TEST = "test"


def pad_label(tensor, size):
    return torch.cat((tensor,(size[1]-1)*torch.ones([size[0]-tensor.size(0)]).long().cuda()),0)
def pad_outs(tensor,size):
    return torch.cat((tensor, torch.rand([(size - tensor.size(0)),tensor.size(1)]).cuda()))

class SummaryWriter:
    def __init__(self, dir, model_desc=''):
        if not os.path.exists(dir):
            os.makedirs(dir)

        self.dir = dir
        self.dict = {}
        if model_desc:
            self.dict['about'] = model_desc

    def add_scalar(self, tag, value, key):
        if tag not in self.dict:
            self.dict[tag] = {}

        self.dict[tag][key] = value

    def commit(self):
        with open(os.path.join(self.dir, "writer.json"), "w+") as f:
            ujson.dump(self.dict, f)
            
def set_seeds(worker_id):
    seed = torch.initial_seed() % 2 ** 31
    np.random.seed(seed + 1)
    random.seed(seed + 2)
    
def create_exp_folder(base_path):
    results = sorted(filter(lambda f: not f.startswith("."), os.listdir(base_path)))
    if len(results) == 0:
        exp_id = 1
    else:
        exp_id = len(results) + 1
    exp_id = str(exp_id)

    os.mkdir(os.path.join(base_path, exp_id))
    return exp_id


def layer_norm_normalization(data: torch.tensor, dim=-1):
    """
    Normalize outputs to norm=1.0
    """
    return data / data.norm(dim=dim, keepdim=True)

In [20]:
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import one_hot
from tqdm import autonotebook, tqdm_notebook as tqdm


def train(model, loss_fn, n_epochs, patience, batch_size, lr, exp_id, num_lines=None):
    #writer = SummaryWriter(os.path.join(OBSERVER_PATH, exp_id), str(model))
    #last_model_path = os.path.join(OBSERVER_PATH, exp_id, "test_last_model.pth")
    #best_model_path = os.path.join(OBSERVER_PATH, exp_id, "test_best_model.pth")
    best_model_path = "test_best_model.pth"
    last_model_path = "test_last_model.pth"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    try:
        #import pdb;pdb.set_trace()
        train_dataset = ImdbDataset_v2(dados,num_lines, train=True)
        val_dataset = ImdbDataset_v2(dados,num_lines)
    #    if len(train_dataset) % batch_size == 1 or len(val_dataset) % batch_size == 1:
    #        raise Exception("Last batch size equal to 1 will cause error")
    #    sampler = RandomSampler(int(len(train_dataset) / 5))
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=batch_size, 
                                      collate_fn=pad_batch,
                                      worker_init_fn=set_seeds,
                                      )
        val_dataloader = DataLoader(val_dataset,
                                    batch_size=batch_size, 
                                    collate_fn=pad_batch, 
                                    worker_init_fn=set_seeds)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        epochs = range(1, n_epochs + 1)
        best_loss = np.inf
        epochs_since_best = 0

        for epoch in epochs:
            epoch_dataloader = train_dataloader


            train_info = run_epoch("train", epoch, model, epoch_dataloader, loss_fn, optimizer, batch_size,
                                   extra_metrics=[accuracy])
            val_info = run_epoch("validation", epoch, model, val_dataloader, loss_fn, optimizer, batch_size,
                                 extra_metrics=[accuracy])
            """
            if val_info["Gini"] > best_loss:
                best_loss = val_info["Gini"]
                torch.save(model, best_model_path)
                epochs_since_best = 0
                print("Saving this version")
            """
            if not torch.isnan(torch.tensor(val_info["loss"])):
                if val_info["loss"] < best_loss:
                    best_loss = val_info["loss"]
                    torch.save(model, best_model_path)
                    epochs_since_best = 0
        #            with open('s3://data-nexa-sets/pdf/distance_model_char.pth', 'wb') as f:
        #                torch.save(model.state_dict(), f)
                else:
                    epochs_since_best += 1
            else:
                epochs_since_best += 1
            torch.save(model, last_model_path, pickle_module=dill)
            if epochs_since_best > patience:
                break

            #writer.commit()
    except Exception as e:
        print(e)
        torch.cuda.empty_cache()  

In [21]:


def accuracy(outputs, targets):
    #import pdb;pdb.set_trace()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    target_mask = (torch.arange(2).unsqueeze(0).expand(targets.size(0), -1).to(device) == targets.unsqueeze(1).to(device)).float()
    lambdas = outputs.exp()
    prob_expo = 1-(-lambdas/2).exp()
    soft = prob_expo.squeeze(1).round()
    comp = targets.to(device) == soft
    return float((comp.float().sum())/len(comp))
def accuracy(outs, targets):
    #import pdb;pdb.set_trace()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    soft = torch.softmax(outs,dim = 1)
    comp = targets.to(device) == torch.argmax(soft,1)
    return float((comp.float().sum())/len(comp))
#def accuracy(outs, targets):
    #import pdb;pdb.set_trace()
#    return abs(float((targets.unsqueeze(1).to(device)-outs).mean()))
def log_softmax(data):
    prob_numerators = torch.exp(data)
    prob_denominators = prob_numerators.sum(-1).unsqueeze(-1)
    return (data - torch.log(prob_denominators))
def numpy_ewma_vectorized(data, window=40):

    data = np.array(data)
    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha

    scale = 1/alpha_rev
    n = data.shape[0]
    if n > 4*window:
        n = 4 * window
        data = data[-n:]

    r = np.arange(n)
    scale_arr = scale**r

    mult = data*scale_arr
    out = mult.sum() / scale_arr.sum()
    return out

In [22]:
import numpy as np
def gini_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    criteria_sort = y_pred.argsort()[::-1]
    observed_sort = y_true.argsort()[::-1]
    n = len(y_true)
    area = y_true[criteria_sort] * (n - np.arange(0, n))
    area2 = y_true[observed_sort] * (n - np.arange(0, n))
    return (area.sum() - n * y_true.sum() / 2) / (area2.sum() - n * y_true.sum() / 2)


In [23]:
from functools import partial
WMA_WINDOW = 1000

def run_epoch(phase, epoch, model, dataloader, loss_fn, optimizer, batch_size, scheduler=None, extra_metrics=None, device=None,
              writer=None):
    training = phase == "train"
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch} - {phase}")
    if extra_metrics is None:
        extra_metrics = {}
    elif isinstance(extra_metrics, list):
        extra_metrics_tmp = {}
        for metric in extra_metrics:
            if isinstance(metric, tuple):
                metric_name, metric_fn = metric
                extra_metrics_tmp[metric_name] = metric_fn
            else:
                extra_metrics_tmp[metric.__name__] = metric
        extra_metrics = extra_metrics_tmp

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    elif isinstance(device, str):
        device = torch.device(device)
    
    model = model.to(device)

    if training:
        model.train()
        mean_fn = partial(numpy_ewma_vectorized, window=int(WMA_WINDOW/batch_size + 1))
    else:
        model.eval()
        mean_fn = np.mean
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    losses = []
    extra_metrics_values = {metric_name: [] for metric_name, _ in extra_metrics.items()}
    for i, data in enumerate(progress_bar):
        optimizer.zero_grad()
        with torch.set_grad_enabled(training):
            elements_data, y = data#
            results = model(elements_data[0].to(device),elements_data[1].to(device))#
            #import pdb;pdb.set_trace()
            #print(results)
            loss = loss_fn(results, y.long().to(device))#
            if training:
                if not torch.isnan(loss):
                    #loss = torch.sqrt(loss)
                    loss.backward()#retain_graph=True)#
                else:
                    continue
                    import pdb;pdb.set_trace()
                optimizer.step()#
                if scheduler is not None:
                    scheduler.step()
            losses.append(loss.item())#
            progress_bar_dict = {f"{phase} loss": mean_fn(losses)}
            for extra_metric_name, extra_metric_fn in extra_metrics.items():
                value_tmp = extra_metric_fn(results,y)#
                extra_metrics_values[extra_metric_name].append(value_tmp)
                progress_bar_dict[f"{phase} {extra_metric_name}"] =mean_fn(extra_metrics_values[extra_metric_name])
            progress_bar.set_postfix(progress_bar_dict)
    #tot_soft = torch.softmax(total_r,axis = 1)
    mean_loss = mean_fn(losses)
    info = {"loss": mean_loss}
    for extra_metric_name, _ in extra_metrics.items():
        info[extra_metric_name] = mean_fn(extra_metrics_values[extra_metric_name])
    #import pdb;pdb.set_trace()
    #lambdas = total_r.exp()
    #prob_expo = 1-(-lambdas/2).exp()    
    #scr = gini_score(total_y.cpu().detach().numpy(), prob_expo.squeeze(1).cpu().detach().numpy())#tot_soft[:,1].cpu().detach().numpy())
    #print("Gini Score: ", scr)
    #if scr<0:
        #import pdb;pdb.set_trace()
    #info['Gini'] = scr
    if writer is not None:
        for metric_name, metric_mean_value in info.items():
            writer.add_scalar(f"{phase}.{metric_name}", metric_mean_value, epoch)
    return info


In [24]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
import torch.nn.functional as F
class SimpleLSTM(nn.Module):
    def __init__(self,size,hidden_size,num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size = size,hidden_size = hidden_size, num_layers = num_layers, bidirectional = True)
        self.output_layer = nn.Linear(2*hidden_size,2)
        self.activation = F.relu
    def forward(self,inputs,seq_len):
        #import pdb;pdb.set_trace()
        packed_input = pack_padded_sequence(inputs.float(),seq_len,enforce_sorted = False)
        tmp, (h_n, c_n) = self.lstm(packed_input)
        lstm_output = torch.cat([h_n[-1], h_n[-2]], -1)
        output = self.output_layer(lstm_output)
        return output


In [44]:

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
import torch.nn.functional as F
class AttentionLSTM(nn.Module):
    def __init__(self,size,hidden_size,num_layers,d_a=250,r1 = 10):
        super().__init__()
        self.lstm = nn.LSTM(input_size = size,hidden_size = hidden_size, num_layers = num_layers, bidirectional = True)
        self.output_layer = nn.Linear(2*hidden_size,2)
        self.activation = F.relu

        self.att = torch.nn.Linear(size*2,d_a) #WS1
        self.att.bias.data.fill_(0) #bias zerao
        self.ws2 = torch.nn.Linear(d_a,r1) #Ws2, if using multiple attention heads
        self.ws2.bias.data.fill_(0)
        self.r1 = r1
    def masked_softmax(self,vector,mask,dim=0): #for the attention module
        for k in range(vector.size(1)):
            vec = torch.cat((torch.softmax(vector[:mask[k],k,:] ,axis = dim),torch.zeros([vector.size(0)-mask[k],self.r1]).to(device)))
            vec = vec.unsqueeze(1)
            try:
                out = torch.cat((out,vec),axis=1)
            except:
                out = vec
        return out
    def forward(self,inputs,seq_len):
        #import pdb;pdb.set_trace()
        #packed_input = pack_padded_sequence(inputs.float(),seq_len,enforce_sorted = False)
        #tmp, (h_n, c_n) = self.lstm(packed_input)
        tmp = self.lstm(inputs.float())

        phrase_1 = F.relu(self.att(tmp[0]))
        phrase_2 = self.ws2(phrase_1)
        phrase_2 = self.masked_softmax(phrase_2,seq_len)
        attention_phrase = phrase_2.transpose(0,1)
        attention_phrase = attention_phrase.transpose(1,2)
        sentence_embeddings = attention_phrase@(tmp[0].transpose(0,1))
        avg_sentence_embeddings = torch.sum(sentence_embeddings,1)/self.r1



        #lstm_output = torch.cat([h_n[-1], h_n[-2]], -1)
        output = self.output_layer(avg_sentence_embeddings)
        return output

In [45]:
#model = ensemble_model(20,2,1)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = AttentionLSTM(size = 256,hidden_size=256,num_layers = 1)
model.to(device)
#with open('s3://data-nexa-sets/pdf/qloss_model.pth', 'rb') as f:
#    model.load_state_dict(torch.load(f, map_location=device))
model.train()


patience = 10
batch_size = 8
lr = 0.001

closs = nn.CrossEntropyLoss()
#floss = FocalLoss(gamma = 2.0)#gamma=2.0)
mseloss = torch.nn.MSELoss()
loss_fn = closs
n_epochs = 30
num_lines = None




In [46]:
train(model, loss_fn, n_epochs, patience, batch_size, lr, "01", num_lines)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


KeyboardInterrupt: ignored

In [1]:
1+1

2